In [2]:
import sys
import numpy as np
import re

In [3]:
# Read in the gene file +/- 1000 base pairs and break it up into 3 pieces
def read_gene_string(genefile):

    # Import Gene -+1000 base pairs - clean in the same way
    with open(genefile, 'r') as file:
        Gene_plus = file.read().replace('\n', '')

    # Clean away numbers and spaces
    Gene_plus=Gene_plus.replace(' ', '') # spaces
    remove_digits=str.maketrans('', '', '0123456789')
    Gene_plus=Gene_plus.translate(remove_digits)   # numbers
    Left_of_gene=Gene_plus[0:1000] # 1000 bp's to the left of the Gene
    Right_of_gene=Gene_plus[(len(Gene_plus)-1000):] # 1000 bp's to the right of the Gene (includes 3' UTR segment)
    Gene=Gene_plus[1000:(len(Gene_plus)-1000)]
    
    return (Left_of_gene, Gene, Right_of_gene)

In [58]:
# Read in and process the enzyme list
def read_enzyme_list(enzymefile):

    with open(enzymefile) as file:
        lines=file.readlines()
        enzymes=[line.rstrip() for line in lines]
        
    rsitelist = []
    enamelist = []
    if (len(enzymes) % 2) == 1:
        print('Enzyme list has an odd number of lines. Enzyme list should be a list of enzyme names and restriction sites in each line.')
    else:
        for linei in range(1, len(enzymes), 2):
            rsite = enzymes[linei]
            ename = enzymes[linei-1]
            # check length of restriction site
            if len(rsite) == 6 or len(rsite) == 8:
                # check whether any non-ACGT characters
                if len(re.sub('[ACGT]', '', rsite)) == 0:
                    #check whether palindromic so can focus on only one strand
                    complement = {'A': 'T', 'C': 'G', 'G': 'C', 'T': 'A'}
                    reverse_complement = "".join(complement.get(base, base) for base in reversed(rsite))
                    if rsite==reverse_complement:
                        # add name only if enzyme already in the list
                        rsiteexists=0 
                        for linej in range(len(rsitelist)):
                            if rsitelist[linej] == rsite:
                                rsiteexists=1
                                enamelist[linej] = enamelist[linej]+', '+ename
                                break
                        #add new entry if not yet there
                        if rsiteexists==0:
                            rsitelist.append(rsite)
                            enamelist.append(ename)
    return (np.array(rsitelist), np.array(enamelist))

In [59]:
    genefile   = './Chris_code/clb2_pm_1000.txt'
    enzymefile = './Chris_code/raw_enzyme_list.txt'
    minhomology = 100

In [115]:
Left_of_gene, Gene, Right_of_gene = read_gene_string(genefile)
rsitelist, enamelist = read_enzyme_list(enzymefile)

### Loop

In [116]:
# Right end of gene and left direction of search (case 1. of sahand graph)
rsite_position_list = np.array([Gene[:-minhomology].rfind(rsite) for rsite in rsitelist])
# rsite_position_list
print(rsitelist[14], Gene[rsite_position_list[14]:rsite_position_list[14]+len(rsitelist[14])])

AGATCT AGATCT


In [114]:
# sort and remove not matched rsites
sorted_inds = np.argsort(-rsite_position_list)
rsite_position_list = rsite_position_list[sorted_inds]
matched_inds = (rsite_position_list != -1)
rsite_position_list = rsite_position_list[matched_inds]

# apply sort and remove inds to rsitelist and enamelist
rsitelist = rsitelist[sorted_inds][matched_inds]
enamelist = enamelist[sorted_inds][matched_inds]

rsitelist, enamelist, rsite_position_list

(array(['TCATGA', 'CAATTG', 'GTATAC', 'CAGCTG', 'GATATC', 'TTCGAA',
        'TTTAAA', 'AATATT', 'AGATCT', 'TGTACA', 'ACTAGT'], dtype='<U8'),
 array(['BspHI', 'MfeI, MfeI-HFÂ®', 'BstZ17I, BstZ17I-HFÂ®',
        'PvuII, PvuII-HFÂ®', 'EcoRV, EcoRV-HFÂ®', 'BstBI', 'DraI',
        'SspI, SspI-HFÂ®', 'BglII', 'BsrGI, BsrGI-HFÂ®', 'SpeI, SpeI-HFÂ®'],
       dtype='<U32'),
 array([1355, 1348, 1289, 1213, 1160, 1103, 1043,  765,  746,   65,   39]))

In [109]:
Gene.rfind('CAATTG')

1348